In [1]:
# Import libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Prepare Dataset
# load data

# relative path to npz files
path = 'Measurements'
file_name = 'output_batch_%d.npz'

# Training
# how many frames to load
frames_num = 10
data_frames = []
data_forces = []
for i in range(frames_num):
    file_path = os.path.join(path,file_name %i)
    data = np.load(file_path)
    data_frames.append(data['frames'])
    data_forces.append(data['forces'])

combined_frames = np.concatenate(data_frames, axis=0)
combined_forces = np.concatenate(data_forces, axis=0)

# Test
# file_path = os.path.join(path, file_name %11)
# test_data = np.load(file_path)

In [3]:
features_train = combined_frames
targets_train = combined_forces[:,2]
# features_test = test_data['frames']
# targets_test = test_data['forces']
# targets_test = targets_test[:,2]

# Normalization
# frames_mean = np.mean(features_train)
# frames_std = np.std(features_train)

# forces_z_mean = np.mean(targets_train)
# forces_z_std = np.std(targets_train)

# features_train = (features_train-frames_mean)/frames_std
# targets_train = (targets_train-forces_z_mean)/forces_z_std

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train)

# create feature and targets tensor for test set. 
# featuresTest = torch.from_numpy(features_test)
# targetsTest = torch.from_numpy(targets_test)

In [4]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_channels, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 9, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size = 4, stride = 4)
        self.conv2 = nn.Conv2d(9, 18, kernel_size = 5, stride = 1, padding = 2)
        
        # RNN
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(18 * 16 * 30, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Convolutional layers
        x = self.pool(nn.ELU()(self.conv1(x)))
        x = self.pool(nn.ELU()(self.conv2(x)))
        
        # Reshape for RNN
        x = torch.reshape(x, (100, 10, 18 * 16 * 30))  # Reshape to (batch_size, seq_len, input_size)
        print("Size of x:", x.size())  # Print size of x
        
        # RNN
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        #print("Size of h0:", h0.size())  # Print size of h0 - Debugging - Obsolete
        out, hn = self.rnn(x, h0)
        
        # Output layer
        out = self.fc(out) 
        out = torch.reshape(out, (1000, 1)) # go back to compare to labels
        return out


In [5]:
# batch_size, epoch and iteration
batch_size = 1000
num_epochs = 5

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
# test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
# test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_channels = 3  # RGB channels
hidden_dim = 200  # hidden layer dimension
layer_dim = 2     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(input_channels, hidden_dim, layer_dim, output_dim)

# Define your loss function
error = nn.MSELoss()

# Define your optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Training
seq_dim = 100
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    print('~~~BEGINNING OF DATASET~~~')
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        # print(images.shape)  # Add this line to check the shape of images - Debugging purposes
        images = images.permute(0, 3, 1, 2)
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(images)
        outputs = torch.squeeze(outputs)
        
        # Calculate loss
        loss = error(outputs, labels.float())
        
        # Backpropagation
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
            
        # Store loss and iteration
        loss_list.append(loss.data) #* (forces_z_std^2)
        # Print Loss
        if count % 1 == 0: # for now print for every iteration
            print('Iteration: {}  Loss: {}'.format(count, loss.data.item())) # * (forces_z_std^2)

~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([100, 10, 8640])
Iteration: 1  Loss: 4.382874011993408
Size of x: torch.Size([100, 10, 8640])
Iteration: 2  Loss: 29.53947639465332
Size of x: torch.Size([100, 10, 8640])
Iteration: 3  Loss: 3.8794803619384766
Size of x: torch.Size([100, 10, 8640])
Iteration: 4  Loss: 3.9011552333831787
Size of x: torch.Size([100, 10, 8640])
Iteration: 5  Loss: 11.054900169372559
Size of x: torch.Size([100, 10, 8640])
Iteration: 6  Loss: 9.62796688079834
Size of x: torch.Size([100, 10, 8640])
Iteration: 7  Loss: 3.55464506149292
Size of x: torch.Size([100, 10, 8640])
Iteration: 8  Loss: 1.0004968643188477
Size of x: torch.Size([100, 10, 8640])
Iteration: 9  Loss: 2.4621083736419678
Size of x: torch.Size([100, 10, 8640])
Iteration: 10  Loss: 4.139577865600586
~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([100, 10, 8640])
Iteration: 11  Loss: 4.493381977081299
Size of x: torch.Size([100, 10, 8640])
Iteration: 12  Loss: 2.222580671310425
Size of x: torch

In [7]:
print(loss_list)
print(np.sqrt(np.mean(loss_list[-10:])))

[tensor(4.3829), tensor(29.5395), tensor(3.8795), tensor(3.9012), tensor(11.0549), tensor(9.6280), tensor(3.5546), tensor(1.0005), tensor(2.4621), tensor(4.1396), tensor(4.4934), tensor(2.2226), tensor(1.2223), tensor(1.2487), tensor(2.0600), tensor(2.7436), tensor(1.8221), tensor(1.0458), tensor(1.0687), tensor(1.3370), tensor(1.7958), tensor(1.3005), tensor(1.2495), tensor(0.9700), tensor(1.1632), tensor(1.6711), tensor(1.3235), tensor(1.0239), tensor(0.9398), tensor(1.0460), tensor(1.3022), tensor(1.0713), tensor(1.2062), tensor(0.9726), tensor(1.0870), tensor(1.4314), tensor(1.0904), tensor(0.9484), tensor(0.9362), tensor(1.0238), tensor(1.1657), tensor(1.0054), tensor(1.2072), tensor(0.9610), tensor(1.0957), tensor(1.4220), tensor(1.0481), tensor(0.9415), tensor(0.9629), tensor(1.0661)]
1.0428618


In [8]:
print(outputs) # Small sanity check

tensor([-1.6003, -1.7937, -1.8790, -2.0598, -1.7151, -2.0651, -2.0095, -1.9034,
        -1.8396, -1.9650, -1.7734, -1.7206, -1.6702, -1.6905, -1.5918, -1.6743,
        -1.3881, -1.3098, -1.6614, -1.6196, -1.7900, -1.5775, -1.7072, -1.9229,
        -1.7543, -1.9563, -1.8537, -1.9506, -1.9106, -1.9426, -1.7179, -1.8091,
        -1.8920, -2.0089, -1.8024, -1.9717, -1.9195, -1.8307, -1.8884, -1.8129,
        -1.4820, -1.5080, -1.6399, -1.6472, -1.4307, -1.5776, -1.5851, -1.7084,
        -1.7957, -1.8191, -1.6695, -1.6784, -1.8468, -2.1363, -1.8025, -1.8080,
        -1.7862, -1.9253, -1.8555, -1.8694, -1.8122, -1.7325, -1.9357, -1.8187,
        -1.6443, -1.7599, -1.2610, -1.6594, -1.6417, -1.5341, -1.4309, -1.5400,
        -1.8200, -1.8791, -1.9626, -1.7221, -1.9307, -1.7624, -1.9269, -1.8967,
        -1.7374, -1.7857, -1.8224, -1.9707, -1.9288, -1.9164, -1.7287, -1.8290,
        -1.7544, -1.7362, -1.5518, -1.4585, -1.6048, -1.6975, -1.5010, -1.6555,
        -1.5630, -1.5473, -1.6018, -1.76

In [9]:
print(labels.float())

tensor([-1.0797, -0.9949, -0.9238, -0.8581, -0.7988, -0.7402, -0.6864, -0.6413,
        -0.6007, -0.5637, -0.5303, -0.5034, -0.4781, -0.4516, -0.4438, -0.4478,
        -0.4525, -0.4622, -0.4775, -0.4831, -0.4824, -0.4824, -0.4840, -0.4829,
        -0.4796, -0.4784, -0.4745, -0.4723, -0.4696, -0.4686, -0.4738, -0.4907,
        -0.5179, -0.5654, -0.6264, -0.6943, -0.7702, -0.8488, -0.9312, -1.0187,
        -1.1152, -1.2284, -1.3486, -1.4846, -1.6458, -1.8335, -2.0371, -2.2583,
        -2.4803, -2.6908, -2.8747, -3.0401, -3.1835, -3.2991, -3.3773, -3.4140,
        -3.4004, -3.3335, -3.2181, -3.0745, -2.9110, -2.7374, -2.5585, -2.3805,
        -2.1968, -2.0178, -1.8431, -1.6839, -1.5378, -1.3989, -1.2665, -1.1307,
        -0.9906, -0.8688, -0.7793, -0.7269, -0.7026, -0.7065, -0.7217, -0.7390,
        -0.7422, -0.7402, -0.7296, -0.7031, -0.6639, -0.6237, -0.5844, -0.5494,
        -0.5165, -0.4928, -0.4744, -0.4658, -0.4598, -0.4682, -0.4804, -0.4992,
        -0.5257, -0.5651, -0.6205, -0.69

Progress Summary: 2 Conv&Pool layers, RNN with 2 hidden states of 200 size, 10 sequence length, 5 epochs: 1.042 RMSE. Best performance by a small margin, albeit on training set.